<a href="https://colab.research.google.com/github/doumsdd/DataCleaning/blob/main/Invoices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Projets : Analysons des factures d'énergie 🌠🌠
  
##Analysons des factures d'énergie 🌠🌠
#1. Introduction
Un des cas d'usage fréquents de l'analyse de données est ce qu'on appelle la détection d'anomalies. Il s'agit en fait de checker, à travers des gros jeux de donnnées, s'il n'y a pas des éléments de la chaîne de production qui dysfonctionnent.

Le contrôle de la facturation est un cas d'école intéressant. Comme la consommation d'énergie n'est pas facilement interprétable par un humain, il peut être utile de vérifier les factures émises par les fournisseurs d'énergie afin de repérer d'éventuelles erreurs et, le cas échéant, permettre à l'entreprise d'économiser de l'argent 💰💰

#2. Le projet
 2.1. La mission
L'opérateur de télécommunications national en Slovaquie fait appel à vous pour contrôler ses factures d'électricité. Il vous explique que les réseaux mobiles (et plus directement, les antennes) sont très consommateurs d'énergie. C'est pourquoi, en tant qu'opérateur, il doit dépenser chaque année de très fortes sommes dans l'électricité.

En revanche, il a le sentiment (et certains de ses salariés lui ont confirmé cela) qu'il y a des erreurs dans les factures et que certaines sommes ne sont pas justifiées. Mais il y a un nombre de factures colossal et son service de comptabilité n'a pas les compétences nécessaires pour traiter autant de données.

Il vous demande donc de récupérer ici le fichier qui regroupe toutes les factures pour l'ensemble des antennes de l'opérateur. Et de le nettoyer pour que les équipes terrain puissent l'utiliser régulièrement, dès qu'un besoin de vérification sera remonté.

Finalement, ton objectif est de transformer le fichier qu'a récupéré la direction en un fichier exploitable. Pour les équipes de comptabilité, "exploitable" signifie qu'ils n'ont qu'à rechercher le nom d'une antenne en particulier, et ils peuvent obtenir l'historique de sa consommation d'énergie pour vérifier s'il y a des anomalies. Pour que cet historique soit intéressant, ils ont besoin d'avoir la consommation moyenne journalière (comme ça par ex, ils peuvent comparer un mois de Février qui fait 28 jours avec un mois de janvier qui fait 31 jours, sans se dire que la diminution vient de la différence de jours dans un mois).

#2.2. Les tâches
1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".

In [2]:
import pandas as pd
# Load the data with correct delimiter
df = pd.read_csv('/content/Invoice_20201220.csv', delimiter='|')

# Display the first few rows of the dataframe
df.head()

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"


In [3]:
import re

# Function to extract the code
def extract_code(site_name):
    match = re.search(r'\b(\d{4}[A-Za-z]{2})\b', site_name)
    return match.group(1) if match else None

# Apply the function to the "SITE_NAME" column
df['SITE_NAME'] = df['SITE_NAME'].apply(extract_code)

df.head()

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"


2) On te demande de supprimer la colonne "INVOICE_NAME" qui n'apporte rien aux équipes.


In [4]:

df = df.drop(columns=['INVOICE_NAME'])

df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,01/05/2018,31/05/2018,2 574,"333,37"


 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année). Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.

In [5]:
df['BEGIN'] = pd.to_datetime(df['BEGIN'], dayfirst=True)
df['END'] = pd.to_datetime(df['END'], dayfirst=True)

df['FACTURATION'] = (df['END'] - df['BEGIN']).dt.days + 1

df.head()


,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,FACTURATION
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",31
1,0001BB,2018-02-01,2018-02-28,2 211,"286,82",28
2,0001BB,2018-03-01,2018-03-31,2 450,"317,32",31
3,0001BB,2018-04-01,2018-04-30,2 431,"315,00",30
4,0001BB,2018-05-01,2018-05-31,2 574,"333,37",31


4) A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.


In [6]:
# ON ENLEVE L'ESPACE DE KWH_PERIOD
df['KWH_PERIOD'] = df['KWH_PERIOD'].str.replace(' ', '').astype(int)

df.head()


,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,FACTURATION
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31


In [7]:
df['DAILY_CONSUMPTION'] = df['KWH_PERIOD'] / df['FACTURATION']

df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,FACTURATION,DAILY_CONSUMPTION
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.935484
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,78.964286
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.032258
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.033333
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.032258


5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.


In [9]:
df['TOTAL_INVOICES'] = df.groupby('SITE_NAME')['SITE_NAME'].transform('count')

df.head(100)
# nombre d'invoices pour chaque antenne

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,FACTURATION,DAILY_CONSUMPTION,TOTAL_INVOICES
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.935484,35.0
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,78.964286,35.0
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.032258,35.0
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.033333,35.0
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.032258,35.0
...,...,...,...,...,...,...,...,...
95,0001KO,2020-02-01,2020-02-29,2377,"321,70",29,81.965517,35.0
96,0001KO,2020-03-01,2020-03-31,2596,"350,71",31,83.741935,35.0
97,0001KO,2020-04-01,2020-04-30,2675,"365,40",30,89.166667,35.0
98,0001KO,2020-05-01,2020-05-31,2781,"375,22",31,89.709677,35.0


6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie.

---



In [12]:
# Calcul du coefficient de variation pour chaque antenne
cv_df = df.groupby('SITE_NAME').agg(
    mean_consumption=('KWH_PERIOD', 'mean'),
    dis_consumption=('KWH_PERIOD', 'std')
).reset_index()

cv_df['COEFF_VARIATION'] = cv_df['dis_consumption'] / cv_df['mean_consumption']

cv_df[['SITE_NAME', 'COEFF_VARIATION']]


,SITE_NAME,COEFF_VARIATION
0,0001BB,0.074126
1,0001BR,0.060068
2,0001KO,0.111631
3,0002BB,0.075073
4,0002BR,0.089522
...,...,...
2397,9513BB,0.003767
2398,9515BB,1.178721
2399,9516BB,0.018127
2400,9517BB,0.000559


In [14]:
# Save the modified dataframe to a new CSV file
df.to_csv('/content/Modified_Invoice_20201220.csv', index=False)

# Save the antenna CV data to a new CSV file
cv_df[['SITE_NAME', 'COEFF_VARIATION']].to_csv('/content/Antenna_Coeff_Variation_20201220.csv', index=False)

